In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import argparse
import os
from pathlib import Path
from fpdf import FPDF



# ============================================================
# 1️⃣ Leitura do arquivo CSV (ajuste o nome conforme o seu)
# ============================================================
df = pd.read_csv("dados_credito_b3.csv", sep=";")

# Mostra as colunas detectadas
print("Colunas detectadas:", list(df.columns))

# ============================================================
# 2️⃣ Normalização dos nomes de colunas (remove espaços e acentos)
# ============================================================
df.columns = (
    df.columns.str.replace(" ", "", regex=False)
               .str.replace(".", "", regex=False)
               .str.replace("í", "i")
               .str.replace("ã", "a")
               .str.replace("ç", "c")
)

# Esperado: ['Papel','EV/EBITDA','MrgEbit','MrgLiq','LiqCorr','ROIC','ROE','Liq2meses','PatrimLiq','DivBrut/Patrim','Cres_Receita5a']

# ============================================================
# 3️⃣ Limpeza e conversão numérica
# ============================================================
cols_num = [c for c in df.columns if c != "Papel"]

for c in cols_num:
    df[c] = (
        df[c].astype(str)
        .str.replace("%", "", regex=False)
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
    )
    df[c] = pd.to_numeric(df[c], errors="coerce")

# ============================================================
# 4️⃣ Cálculo de indicadores de crédito / robustez financeira
# ============================================================

# Criando alguns indicadores derivados
df["EndividamentoRelativo"] = df["DivBrut/Patrim"]
df["LiquidezGeral"] = df["LiqCorr"]
df["RentabilidadeMedia"] = df[["ROIC", "ROE"]].mean(axis=1)
df["MargemMedia"] = df[["MrgEbit", "MrgLiq"]].mean(axis=1)
df["Crescimento5a"] = df["Cres_Receita5a"]

# ============================================================
# 5️⃣ Função de Score e Rating
# ============================================================
def calcular_score(row):
    # Normalizações entre 0 e 1
    def norm(x, min_v, max_v):
        return min(max((x - min_v) / (max_v - min_v), 0), 1)

    score = (
        0.25 * norm(row["LiquidezGeral"], 0.5, 3) +
        0.20 * norm(1 - row["EndividamentoRelativo"], 0, 1) +
        0.25 * norm(row["RentabilidadeMedia"], 0, 15) +
        0.15 * norm(row["MargemMedia"], 0, 10) +
        0.15 * norm(row["Crescimento5a"], -10, 20)
    ) * 100

    # Rating
    if score >= 80:
        rating = "A"
    elif score >= 65:
        rating = "B"
    elif score >= 50:
        rating = "C"
    else:
        rating = "D"

    return round(score, 1), rating

df["Score"], df["Rating"] = zip(*df.apply(calcular_score, axis=1))


In [ ]:
# Ordena as empresas por score (do maior para o menor)
df_sorted = df.sort_values('Score', ascending=False)

# Imprime os resultados
print("\nAnálise de Crédito das Empresas:")
print("="*50)
for _, row in df_sorted.iterrows():
    print(f"\nEmpresa: {row['Papel']}")
    print(f"Score: {row['Score']}")
    print(f"Rating: {row['Rating']}")
    print("-"*30)

In [ ]:
# Análise de Crédito das Empresas

## Classificação dos Ratings

##** Rating A**: Score ≥ 80 - Excelente situação financeira
##** Rating B**: Score ≥ 65 - Boa situação financeira
##** Rating C**: Score ≥ 50 - Situação financeira regular
##** Rating D**: Score < 50 - Situação financeira que requer atenção

In [ ]:
# Formata a tabela com os resultados
resultado_df = df_sorted[['Papel', 'Score', 'Rating']].copy()

# Aplica estilo na tabela
def highlight_rating(val):
    if val == 'A':
        color = '#90EE90'  # Verde claro
    elif val == 'B':
        color = '#FFFFE0'  # Amarelo claro
    elif val == 'C':
        color = '#FFB6C1'  # Rosa claro
    else:
        color = '#FF7F7F'  # Vermelho claro
    return f'background-color: {color}'

# Formata e exibe a tabela
styled_df = resultado_df.style\
    .format({'Score': '{:.1f}'})\
    .applymap(highlight_rating, subset=['Rating'])\
    .set_properties(**{'text-align': 'center'})\
    .set_table_styles([
        {'selector': 'th',
         'props': [('background-color', '#4CAF50'),
                  ('color', 'white'),
                  ('font-weight', 'bold'),
                  ('text-align', 'center')]},
        {'selector': 'td',
         'props': [('padding', '8px'),
                  ('border', '1px solid #ddd')]},
    ])

display(styled_df)

In [ ]:
# Criar diretórios se não existirem
output_dir = "output_credito_pdf"
excel_dir = os.path.join(output_dir, "excel")
pdf_dir = os.path.join(output_dir, "pdf")

os.makedirs(excel_dir, exist_ok=True)
os.makedirs(pdf_dir, exist_ok=True)

# ============================================================
# Exportar para Excel
# ============================================================
excel_path = os.path.join(excel_dir, "analise_credito.xlsx")

# Criar um arquivo Excel com duas sheets
with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    # Sheet 1: Resultados
    resultado_df.to_excel(writer, sheet_name='Resultados', index=False)
    workbook = writer.book
    worksheet = writer.sheets['Resultados']
    
    # Formatar cabeçalho
    header_format = workbook.add_format({
        'bold': True,
        'bg_color': '#4CAF50',
        'font_color': 'white',
        'align': 'center',
        'border': 1
    })
    
    # Aplicar formato no cabeçalho
    for col_num, value in enumerate(resultado_df.columns.values):
        worksheet.write(0, col_num, value, header_format)
    
    # Formatar células com cores para ratings
    rating_formats = {
        'A': workbook.add_format({'bg_color': '#90EE90', 'align': 'center'}),
        'B': workbook.add_format({'bg_color': '#FFFFE0', 'align': 'center'}),
        'C': workbook.add_format({'bg_color': '#FFB6C1', 'align': 'center'}),
        'D': workbook.add_format({'bg_color': '#FF7F7F', 'align': 'center'})
    }
    
    # Aplicar formatos nas células
    for row_num, rating in enumerate(resultado_df['Rating'], start=1):
        worksheet.write(row_num, 2, rating, rating_formats[rating])
    
    # Ajustar largura das colunas
    worksheet.set_column('A:C', 15)
    
    # Sheet 2: Legenda
    legend_data = {
        'Rating': ['A', 'B', 'C', 'D'],
        'Score': ['>= 80', '>= 65', '>= 50', '< 50'],
        'Descrição': [
            'Excelente situação financeira',
            'Boa situação financeira',
            'Situação financeira regular',
            'Situação financeira que requer atenção'
        ]
    }
    legend_df = pd.DataFrame(legend_data)
    legend_df.to_excel(writer, sheet_name='Legenda', index=False)
    worksheet_legend = writer.sheets['Legenda']
    
    # Formatar cabeçalho da legenda
    for col_num, value in enumerate(legend_df.columns.values):
        worksheet_legend.write(0, col_num, value, header_format)
    
    # Aplicar cores na coluna de Rating da legenda
    for row_num, rating in enumerate(legend_df['Rating'], start=1):
        worksheet_legend.write(row_num, 0, rating, rating_formats[rating])
    
    # Ajustar largura das colunas da legenda
    worksheet_legend.set_column('A:A', 10)
    worksheet_legend.set_column('B:B', 10)
    worksheet_legend.set_column('C:C', 35)

# ============================================================
# Exportar para PDF
# ============================================================
pdf_path = os.path.join(pdf_dir, "analise_credito.pdf")

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 15)
        self.cell(0, 10, 'Análise de Crédito das Empresas', 0, 1, 'C')
        self.ln(10)
        
    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Página {self.page_no()}', 0, 0, 'C')

# Criar PDF
pdf = PDF()
pdf.add_page()

# Adicionar tabela de resultados
pdf.set_font('Arial', 'B', 12)
pdf.cell(0, 10, 'Resultados da Análise', 0, 1, 'L')
pdf.ln(5)

# Cabeçalho da tabela
pdf.set_fill_color(76, 175, 80)  # Verde (#4CAF50)
pdf.set_text_color(255, 255, 255)  # Branco
pdf.set_font('Arial', 'B', 10)

col_widths = [60, 30, 30]
for i, col in enumerate(['Empresa', 'Score', 'Rating']):
    pdf.cell(col_widths[i], 10, col, 1, 0, 'C', True)
pdf.ln()

# Conteúdo da tabela
pdf.set_text_color(0, 0, 0)  # Preto
pdf.set_font('Arial', '', 10)

for _, row in resultado_df.iterrows():
    pdf.cell(col_widths[0], 10, str(row['Papel']), 1, 0, 'L')
    pdf.cell(col_widths[1], 10, f"{row['Score']:.1f}", 1, 0, 'C')
    
    # Definir cor de fundo baseado no rating
    if row['Rating'] == 'A':
        pdf.set_fill_color(144, 238, 144)  # Verde claro
    elif row['Rating'] == 'B':
        pdf.set_fill_color(255, 255, 224)  # Amarelo claro
    elif row['Rating'] == 'C':
        pdf.set_fill_color(255, 182, 193)  # Rosa claro
    else:
        pdf.set_fill_color(255, 127, 127)  # Vermelho claro
    
    pdf.cell(col_widths[2], 10, row['Rating'], 1, 0, 'C', True)
    pdf.ln()

# Adicionar legenda
pdf.add_page()
pdf.set_font('Arial', 'B', 12)
pdf.cell(0, 10, 'Legenda dos Ratings', 0, 1, 'L')
pdf.ln(5)

# Cabeçalho da legenda
pdf.set_fill_color(76, 175, 80)
pdf.set_text_color(255, 255, 255)
pdf.set_font('Arial', 'B', 10)

legend_widths = [30, 30, 100]
for i, col in enumerate(['Rating', 'Score', 'Descrição']):
    pdf.cell(legend_widths[i], 10, col, 1, 0, 'C', True)
pdf.ln()

# Conteúdo da legenda
pdf.set_text_color(0, 0, 0)
pdf.set_font('Arial', '', 10)

legend_data = {
    'A': {'score': '>= 80', 'desc': 'Excelente situação financeira', 'color': (144, 238, 144)},
    'B': {'score': '>= 65', 'desc': 'Boa situação financeira', 'color': (255, 255, 224)},
    'C': {'score': '>= 50', 'desc': 'Situação financeira regular', 'color': (255, 182, 193)},
    'D': {'score': '< 50', 'desc': 'Situação financeira que requer atenção', 'color': (255, 127, 127)}
}

for rating, info in legend_data.items():
    pdf.set_fill_color(*info['color'])
    pdf.cell(legend_widths[0], 10, rating, 1, 0, 'C', True)
    pdf.cell(legend_widths[1], 10, info['score'], 1, 0, 'C')
    pdf.cell(legend_widths[2], 10, info['desc'], 1, 0, 'L')
    pdf.ln()

# Salvar PDF
pdf.output(pdf_path)

print(f"\nArquivos gerados com sucesso:")
print(f"Excel: {excel_path}")
print(f"PDF: {pdf_path}")

# 📊 Análise de Crédito B3

Este notebook realiza uma análise de crédito automatizada para empresas listadas na B3, gerando ratings baseados em indicadores financeiros.

## 📋 Funcionalidades

- Leitura e processamento de dados financeiros
- Cálculo de scores baseado em múltiplos indicadores
- Sistema de rating (A, B, C, D) com critérios claros
- Geração de relatórios em Excel e PDF
- Visualização com tabelas formatadas e código de cores

## 🔍 Indicadores Utilizados

- Liquidez Geral
- Endividamento Relativo
- Rentabilidade Média (ROIC e ROE)
- Margem Média (EBIT e Líquida)
- Crescimento em 5 anos

## 📈 Sistema de Rating

- **Rating A** (Score >= 80): Excelente situação financeira
- **Rating B** (Score >= 65): Boa situação financeira
- **Rating C** (Score >= 50): Situação financeira regular
- **Rating D** (Score < 50): Situação financeira que requer atenção

## 📦 Requisitos

```python
pip install pandas numpy matplotlib fpdf xlsxwriter
```

## 🚀 Como Usar

1. Prepare seu arquivo CSV com os dados das empresas
2. Execute o notebook por seções
3. Os relatórios serão gerados na pasta `output_credito_pdf`

## 📂 Estrutura de Arquivos

```
└── output_credito_pdf/
    ├── excel/
    │   └── analise_credito.xlsx
    └── pdf/
        └── analise_credito.pdf
```

## 📝 Formato do CSV de Entrada

O arquivo CSV deve conter as seguintes colunas:
- Papel
- EV/EBITDA
- MrgEbit
- MrgLiq
- LiqCorr
- ROIC
- ROE
- Liq2meses
- PatrimLiq
- DivBrut/Patrim
- Cres_Receita5a